In [ ]:

!pip install opentrons
import opentrons
import opentrons.execute
from opentrons import protocol_api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Serial Dilution 

In [ ]:
metadata = {
    'apiLevel': '2.13',
    'protocolName': 'Serial Dilution Tutorial',
    'description': '''This protocol is the outcome of following the
                   Python Protocol API Tutorial located at
                   https://docs.opentrons.com/v2/tutorial.html. It takes a
                   solution and progressively dilutes it by transferring it
                   stepwise across a plate.''',
    'author': 'New API User'
    }

In [ ]:
def run(protocol: protocol_api.ProtocolContext):
    metadata = {'apiLevel': '2.0'}
    plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
    tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
    p300 = protocol.load_instrument('p300_multi_gen2', 'right',
    tip_racks=[tiprack_1])
    reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)

    row = plate.rows()[0]
    p300.pick_up_tip()
    p300.transfer(100, reservoir['A1'], row[1:], new_tip='never')
    p300.blow_out()
    p300.transfer(200, reservoir['A2'], row[0], new_tip='never')
    p300.blow_out()
    p300.transfer(100, row[:10], row[1:11], mix_after=(3, 50), new_tip='never', blow_out=True)
    p300.aspirate(100, row[10])
    p300.drop_tip()       

In [ ]:
from opentrons import simulate
metadata = {'apiLevel': '2.0'}
protocol = simulate.get_protocol_api('2.0')
plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
plate2 = protocol.load_labware('corning_96_wellplate_360ul_flat', 4)
tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
p300 = protocol.load_instrument('p300_multi_gen2', 'right',
tip_racks=[tiprack_1])
reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)
#Reservoirs: A1 is PBS, A2 is Fluroesciein, A3 Sofarhodamine Red, A4 Blue
row = plate.rows()[0]
p300.pick_up_tip()
p300.transfer(100, reservoir['A1'], row[1:5], new_tip='never')
p300.transfer(100, reservoir['A1'], row[6:10], new_tip='never')
p300.blow_out()
p300.transfer(200, reservoir['A2'], row[0], new_tip='never')
p300.blow_out()
p300.transfer(100, row[:3], row[1:4], mix_after=(3, 50), new_tip='never', blow_out=True)
p300.aspirate(100, row[3])
p300.drop_tip()     
p300.pick_up_tip()
p300.transfer(200, reservoir['A3'], row[5], new_tip='never')
p300.blow_out()
p300.transfer(100, row[5:8], row[6:9], mix_after=(3, 50), new_tip='never', blow_out=True)
p300.aspirate(100, row[8])
p300.drop_tip() 
p300.pick_up_tip()    
p300.transfer(100, reservoir['A1'], row[0:5], new_tip='never')
p300.transfer(100, reservoir['A1'], row[5:10], new_tip='never')
p300.drop_tip()  


row = plate2.rows()[0]
p300.pick_up_tip()
p300.transfer(100, reservoir['A1'], row[1:5], new_tip='never')
p300.transfer(100, reservoir['A1'], row[6:10], new_tip='never')
p300.blow_out()
p300.transfer(200, reservoir['A2'], row[0], new_tip='never')
p300.blow_out()
p300.transfer(100, row[:3], row[1:4], mix_after=(3, 50), new_tip='never', blow_out=True)
p300.aspirate(100, row[3])
p300.drop_tip()     
p300.pick_up_tip()
p300.transfer(200, reservoir['A4'], row[5], new_tip='never')
p300.blow_out()
p300.transfer(100, row[5:8], row[6:9], mix_after=(3, 50), new_tip='never', blow_out=True)
p300.aspirate(100, row[8])
p300.drop_tip() 
p300.pick_up_tip()    
p300.transfer(100, reservoir['A1'], row[0:5], new_tip='never')
p300.transfer(100, reservoir['A1'], row[5:10], new_tip='never')
p300.drop_tip()  


for line in protocol.commands():
    print(line)


Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2
Transferring 100.0 from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 to A2 of Corning 96 Well Plate 360 µL Flat on 1
Aspirating 100.0 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 at 94.0 uL/sec
Dispensing 100.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 1 at 94.0 uL/sec
Aspirating 100.0 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 at 94.0 uL/sec
Dispensing 100.0 uL into A3 of Corning 96 Well Plate 360 µL Flat on 1 at 94.0 uL/sec
Aspirating 100.0 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 at 94.0 uL/sec
Dispensing 100.0 uL into A4 of Corning 96 Well Plate 360 µL Flat on 1 at 94.0 uL/sec
Aspirating 100.0 uL from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 at 94.0 uL/sec
Dispensing 100.0 uL into A5 of Corning 96 Well Plate 360 µL Flat on 1 at 94.0 uL/sec
Transferring 100.0 from A1 of USA Scientific 12 Well Reservoir 22 mL on 3 to A7 of Corning 96 Well Plate 360 µL Flat o

# MoClo

In [ ]:
from opentrons import protocol_api, simulate
import math
import numpy as np
import csv


# Specify arguments here
# Volume of each DNA part to use in assembly (75-100ng optimal)
part_volume = 1
# If set to true, will use 1 tip per part, and wash between each aspiration/dispensation
reuse_tips = False
# Specify reservoir format. Names must be unique. Each well in the assembly plate will contain
# one DNA part for each group. MM (master mix) will be added to each well, and should contain 
# everything except the DNA parts (i.e. water and GG master mix in the correct ratio, and also 
# opetionally the destination vector). This csv_raw can be generated by opening a csv file with 
# a text editor and copy-pasting it here. MM, wash1, and wash2 are protected variables and 
# cannot be used for part or group names - only use for master mix and wash reservoirs.
csv_raw = '''
A_wt,A_mut,B_wt,B_mut,C_wt,C_mut,D_wt,D_mut,MM,wash1,wash2
1,1,2,2,3,3,4,4,MM,wash1,wash2
A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12
'''

csv_data = csv_raw.splitlines()[1:] # Discard the blank first line.
csv_data = [i.split(',') for i in csv_data]

def cartesian(arrays, out=None):
    
    '''function generates all DNA combinations such that every combination contains
    one part from each group, and every group is represented'''
    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = int(n / arrays[0].size) 
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m, 1:])
        for j in range(1, arrays[0].size):
        #for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m, 1:] = out[0:m, 1:]
    return out


row_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}

metadata = {'apiLevel': '2.7'}
def run(protocol: protocol_api.ProtocolContext):
    
    # Creates dictionary of name: (group, well) from csv
    myDict = {}
    for name, group, well in zip(csv_data[0], csv_data[1], csv_data[2]):
        myDict[name] = (group, well)

    # Creates dictionary of group: [list of names in group]
    group_dict = {}
    for name, tup in myDict.items():
        if tup[0] in group_dict.keys():
            group_dict[tup[0]].append(name)
        else:
            group_dict[tup[0]] = [name]

    # Generates all combinations using cartesian from group_dict
    combs = cartesian((gr_list for gr_list in group_dict.values()))

    # Specify special wells
    MM_well = myDict['MM'][1]
    if reuse_tips:
        wash_1 = myDict['wash1'][1]
        wash_2 = myDict['wash2'][1]

    # Create dictionary with the wells to pipette into for each DNA sequence
    well_dict = {name: [] for name in csv_data[0] if name not in ['MM', 'wash1', 'wash2']}
    for i, comb in enumerate(combs):
        for name in csv_data[0]:
            if name in comb and name not in ['MM', 'wash1', 'wash2']:
                well_dict[name].append(i)

    # Initialise labware
    reservoir = protocol.load_labware('4ti0136_96_wellplate_2200ul', 3)
    tc_mod = protocol.load_module('Thermocycler Module')
    plate_thermo = tc_mod.load_labware('4ti0960rig_96_wellplate_200ul')
    tips20 = protocol.load_labware('opentrons_96_tiprack_20ul', 2)

    p20 = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tips20])

    tc_mod.open_lid()

    # Calculate volume of master mix in each well for a total volume of 20 ul
    MM_vol = 20 - len(group_dict)
    
    # Pipette MM into each well
    p20.pick_up_tip()
    for i in range(len(combs)):
        well_loc = row_dict[math.floor(i/12)] + str(i%12 + 1)
        p20.aspirate(MM_vol, reservoir[MM_well].bottom(0.5))
        p20.dispense(MM_vol, plate_thermo[well_loc])
        p20.blow_out()
    p20.drop_tip()


    # Move parts to wells in all combinations
    for name, well_list in well_dict.items():
        if reuse_tips:
            p20.pick_up_tip()
            for j in well_list:
                well_loc = row_dict[math.floor(j/12)] + str(j%12 + 1)
                p20.aspirate(part_volume, reservoir[myDict[name][1]].bottom(0.5))
                p20.dispense(part_volume, plate_thermo[well_loc])
                p20.blow_out()
                
                # Wash steps allow tip recycling
                p20.mix(2, 5, reservoir[wash_1].bottom(0.5))
                p20.blow_out()
                p20.mix(2, 5, reservoir[wash_2].bottom(0.5))
                p20.blow_out()
            p20.drop_tip()
        else:
            for j in well_list:
                well_loc = row_dict[math.floor(j/12)] + str(j%12 + 1)
                p20.transfer(1, reservoir[myDict[name][1]].bottom(0.5), plate_thermo[well_loc], blow_out=True)

    tc_mod.close_lid()
    tc_mod.set_lid_temperature(temperature=75)

    # GoldenGate protocol
    for i in range(30):
        tc_mod.set_block_temperature(37, hold_time_minutes=1)
        tc_mod.set_block_temperature(16, hold_time_minutes=1)
    tc_mod.set_block_temperature(60, hold_time_minutes=5)
    tc_mod.set_block_temperature(4, hold_time_minutes=5)
    tc_mod.open_lid()
    tc_mod.deactivate()

In [ ]:
row_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H'}
part_volume = 1
reuse_tips = False
csv_raw = '''
A_wt,A_mut,B_wt,B_mut,C_wt,C_mut,D_wt,D_mut,MM
1,1,2,2,3,3,4,4,MM
A1,A2,A3,A4,A5,A6,A7,A8,A9,A10
'''

csv_data = csv_raw.splitlines()[1:] # Discard the blank first line.
csv_data = [i.split(',') for i in csv_data]



protocol = simulate.get_protocol_api('2.7')
myDict = {}
for name, group, well in zip(csv_data[0], csv_data[1], csv_data[2]):
    myDict[name] = (group, well)

group_dict = {}
for name, tup in myDict.items():
    if tup[0] in group_dict.keys():
        group_dict[tup[0]].append(name)
    else:
        group_dict[tup[0]] = [name]

combs = cartesian((gr_list for gr_list in group_dict.values()))

MM_well = myDict['MM'][1]
if reuse_tips:
    wash_1 = myDict['wash1'][1]
    wash_2 = myDict['wash2'][1]

well_dict = {name: [] for name in csv_data[0] if name not in ['MM', 'wash1', 'wash2']}
for i, comb in enumerate(combs):
    for name in csv_data[0]:
        if name in comb and name not in ['MM', 'wash1', 'wash2']:
            well_dict[name].append(i)

# Initialise labware
reservoir = protocol.load_labware('4ti0136_96_wellplate_2200ul', 3)
tc_mod = protocol.load_module('Thermocycler Module')
plate_thermo = tc_mod.load_labware('4ti0960rig_96_wellplate_200ul')
tips20 = protocol.load_labware('opentrons_96_tiprack_20ul', 2)

p20 = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tips20])

tc_mod.open_lid()

# Calculate water in each well
MM_vol = 20 - len(group_dict)
p20.pick_up_tip()
for i in range(len(combs)):
    well_loc = row_dict[math.floor(i/12)] + str(i%12 + 1)
    p20.aspirate(MM_vol, reservoir[MM_well].bottom(0.5))
    p20.dispense(MM_vol, plate_thermo[well_loc])
    p20.blow_out()
p20.drop_tip()


# Move parts to wells in all combinations
for name, well_list in well_dict.items():
    if reuse_tips:
        p20.pick_up_tip()
        for j in well_list:
            well_loc = row_dict[math.floor(j/12)] + str(j%12 + 1)
            p20.aspirate(part_volume, reservoir[myDict[name][1]].bottom(0.5))
            p20.dispense(part_volume, plate_thermo[well_loc])
            p20.blow_out()
            # Wash steps allow tip recycling
            p20.mix(2, 5, reservoir[wash_1].bottom(0.5))
            p20.blow_out()
            p20.mix(2, 5, reservoir[wash_2].bottom(0.5))
            p20.blow_out()
        p20.drop_tip()
    else:
        for j in well_list:
            well_loc = row_dict[math.floor(j/12)] + str(j%12 + 1)
            p20.transfer(1, reservoir[myDict[name][1]].bottom(0.5), plate_thermo[well_loc], blow_out=True)

tc_mod.close_lid()
tc_mod.set_lid_temperature(temperature=75)

for i in range(30):
    tc_mod.set_block_temperature(37, hold_time_minutes=1)
    tc_mod.set_block_temperature(16, hold_time_minutes=1)
tc_mod.set_block_temperature(60, hold_time_minutes=5)
tc_mod.set_block_temperature(4, hold_time_minutes=5)
tc_mod.open_lid()
tc_mod.deactivate()

for line in protocol.commands():
    print(line)

Opening Thermocycler lid
Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 2
Aspirating 15.0 uL from A9 of NEST 12 Well Reservoir 15 mL on 3 at 7.56 uL/sec
Dispensing 15.0 uL into A1 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 7.56 uL/sec
Blowing out at A1 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7
Aspirating 15.0 uL from A9 of NEST 12 Well Reservoir 15 mL on 3 at 7.56 uL/sec
Dispensing 15.0 uL into A2 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 7.56 uL/sec
Blowing out at A2 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7
Aspirating 15.0 uL from A9 of NEST 12 Well Reservoir 15 mL on 3 at 7.56 uL/sec
Dispensing 15.0 uL into A3 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 7.56 uL/sec
Blowing out at A3 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7
Aspirating 15.0 uL from A9 of NEST 12 Well Reservoir 15 mL on 3 at 7.56 uL/sec
Dispensing 15.0 uL

## Thermocycler

In [ ]:
#Thermocycler Protocol for when in use with real cells
from opentrons import simulate
metadata = {'apiLevel': '2.7'}
protocol = simulate.get_protocol_api('2.7')
plate = protocol.load_labware('nest_96_wellplate_200ul_flat', 1)
tc_mod = protocol.load_module('Thermocycler Module')
#plate2 = protocol.load_labware('nest_96_wellplate_200ul_flat', 4)
plate2 = tc_mod.load_labware('nest_96_wellplate_200ul_flat')
tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
p300 = protocol.load_instrument('p300_single_gen2', 'left',
tip_racks=[tiprack_1])
reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)
#Reservoirs: A1 is PBS, A2 is Fluroesciein, A3 Sofarhodamine Red, A4 Blue
p300.pick_up_tip()

for i in range(8):
    rowdest = plate2.rows()[i]
    p300.transfer(25, reservoir["A8"], rowdest[0:8], new_tip='never')
p300.drop_tip() 

for i in range(8):
    row = plate.rows()[i]
    rowdest = plate2.rows()[i]
    p300.transfer(20, row[0:8], rowdest[0:8], new_tip='always')

tc_mod.close_lid() 

tc_mod.set_block_temperature(4, hold_time_minutes=30)
tc_mod.set_block_temperature(42, hold_time_minutes=.75)
tc_mod.set_block_temperature(4, hold_time_minutes=5)
tc_mod.open_lid()

for line in protocol.commands():
    print(line)

from opentrons import simulate
metadata = {'apiLevel': '2.8'}
protocol = simulate.get_protocol_api('2.8')
plate = protocol.load_labware('nest_96_wellplate_200ul_flat', 1)
tc_mod = protocol.load_module('Thermocycler Module')
#plate2 = protocol.load_labware('nest_96_wellplate_200ul_flat', 4)
plate2 = tc_mod.load_labware('nest_96_wellplate_200ul_flat')
tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
p300 = protocol.load_instrument('p300_single_gen2', 'left',
tip_racks=[tiprack_1])
reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)
for i in range(8):
    rowdest = plate2.rows()[i]
    p300.transfer(150, reservoir["A7"], rowdest[0:8], new_tip='always')

tc_mod.close_lid() 
tc_mod.set_block_temperature(37, hold_time_minutes=60)
tc_mod.open_lid()

for line in protocol.commands():
    print(line)

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2
Transferring 25.0 from A8 of USA Scientific 12 Well Reservoir 22 mL on 3 to A1 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7
Aspirating 25.0 uL from A8 of USA Scientific 12 Well Reservoir 22 mL on 3 at 92.86 uL/sec
Dispensing 25.0 uL into A1 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Aspirating 25.0 uL from A8 of USA Scientific 12 Well Reservoir 22 mL on 3 at 92.86 uL/sec
Dispensing 25.0 uL into A2 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Aspirating 25.0 uL from A8 of USA Scientific 12 Well Reservoir 22 mL on 3 at 92.86 uL/sec
Dispensing 25.0 uL into A3 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Aspirating 25.0 uL from A8 of USA Scientific 12 Well Reservoir 22 mL on 3 at 92.86 uL/sec
Dispensing 25.0 uL into A4 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 u

In [ ]:
import opentrons
import opentrons.execute
from opentrons import protocol_api


metadata = {'apiLevel': '2.7'}
def run(protocol: protocol_api.ProtocolContext):
    plate = protocol.load_labware('nest_96_wellplate_200ul_flat', 1)
    tc_mod = protocol.load_module('Thermocycler Module')
  #plate2 = protocol.load_labware('nest_96_wellplate_200ul_flat', 4)
    plate2 = tc_mod.load_labware('nest_96_wellplate_200ul_flat')
    tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
    p300 = protocol.load_instrument('p300_single_gen2', 'left',
    tip_racks=[tiprack_1])
    reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)
  #Reservoirs: A1 is PBS, A2 is Fluroesciein, A3 Sofarhodamine Red, A4 Blue
    p300.pick_up_tip()

    for i in range(8):
        rowdest = plate2.rows()[i]
        p300.transfer(25, reservoir["A8"], rowdest[0:8], new_tip='never')
    p300.drop_tip() 

    for i in range(8):
        row = plate.rows()[i]
        rowdest = plate2.rows()[i]
        p300.transfer(25, row[0:8], rowdest[0:8], new_tip='always')

    tc_mod.close_lid() 

    tc_mod.set_block_temperature(4, hold_time_minutes=30)
    tc_mod.set_block_temperature(42, hold_time_minutes=.75)
    tc_mod.set_block_temperature(4, hold_time_minutes=5)
    tc_mod.open_lid()

    for i in range(8):
        rowdest = plate2.rows()[i]
        p300.transfer(150, reservoir["A7"], rowdest[0:8], new_tip='always')

    tc_mod.close_lid() 
    tc_mod.set_block_temperature(37, hold_time_minutes=60)
    tc_mod.open_lid()



In [ ]:
# Faster Version of Script for Dye Runs with pipette reusage
from opentrons import simulate
metadata = {'apiLevel': '2.7'}
protocol = simulate.get_protocol_api('2.7')
plate = protocol.load_labware('nest_96_wellplate_200ul_flat', 1)
tc_mod = protocol.load_module('Thermocycler Module')
plate2 = tc_mod.load_labware('nest_96_wellplate_200ul_flat')
tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
p300 = protocol.load_instrument('p300_single_gen2', 'left',
tip_racks=[tiprack_1])
reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)
p300.pick_up_tip()
tot = 0
# Calculation of amount to be dispensed each time.
# n1 is a multiple of dispensed amount * total wells therefore equals total volume required
n1 = 25*64
for i in range(1, n1+1):
    if n1%i == 0 and i <= 300 and i%25 == 0:
    totvol = i

tot = 0
for j in range(1,9):
    for i in range(1,9):
        if tot == 0:
            p300.aspirate(totvol, reservoir['A7'])
            tot = totvol
    p300.dispense(25, plate2[(chr(ord('@')+j)+str(i))])
    tot -= 25
if tot != 0:
    p300.dispense(tot, reservoir['A7'])
p300.drop_tip()  
p300.pick_up_tip()

for i in range(8):
    row = plate.rows()[i]
    rowdest = plate2.rows()[i]
    p300.transfer(25, row[0:8], rowdest[0:8], new_tip='never',blow_out=True)
    p300.blow_out()
p300.drop_tip()  

tc_mod.close_lid() 

tc_mod.set_block_temperature(4, hold_time_minutes=30)
tc_mod.set_block_temperature(42, hold_time_minutes=.75)
tc_mod.set_block_temperature(4, hold_time_minutes=5)
tc_mod.open_lid()

p300.pick_up_tip()
#for i in range(8):
#  rowdest = plate2.rows()[i]
#  p300.transfer(150, reservoir["A8"], rowdest[0:8], new_tip='never',blow_out=True,blowout_location='destination well')
#p300.drop_tip()
tot = 0
n1 = 150*64
for i in range(1, n1+1):
if n1%i == 0 and i <= 300 and i%150 == 0:
    totvol = i
for j in range(1,9):
    for i in range(1,9):
        if tot == 0:
            p300.aspirate(totvol, reservoir['A8'])
            tot = totvol
        p300.dispense(150, plate2[(chr(ord('@')+j)+str(i))])
        tot -= 150

p300.drop_tip()  
tc_mod.close_lid() 
tc_mod.set_block_temperature(37, hold_time_minutes=60)
tc_mod.open_lid()

for line in protocol.commands():
    print(line)




Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 2
Aspirating 200.0 uL from A7 of USA Scientific 12 Well Reservoir 22 mL on 3 at 92.86 uL/sec
Dispensing 25.0 uL into A1 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Dispensing 25.0 uL into A2 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Dispensing 25.0 uL into A3 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Dispensing 25.0 uL into A4 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Dispensing 25.0 uL into A5 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Dispensing 25.0 uL into A6 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Dispensing 25.0 uL into A7 of NEST 96 Well Plate 200 µL Flat on Thermocycler Module GEN1 on 7 at 92.86 uL/sec
Dispensing 25.0 uL into A8 of NEST 96 Well Plate 200 µL Flat on Thermocycler Mo

In [ ]:
import opentrons
import opentrons.execute
from opentrons import protocol_api


metadata = {'apiLevel': '2.7'}
def run(protocol: protocol_api.ProtocolContext):

    protocol = simulate.get_protocol_api('2.7')
    plate = protocol.load_labware('nest_96_wellplate_200ul_flat', 1)
    tc_mod = protocol.load_module('Thermocycler Module')
    plate2 = tc_mod.load_labware('nest_96_wellplate_200ul_flat')
    tiprack_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 2)
    p300 = protocol.load_instrument('p300_single_gen2', 'left',
    tip_racks=[tiprack_1])
    reservoir = protocol.load_labware('usascientific_12_reservoir_22ml', 3)
    p300.pick_up_tip()
    tot = 0
  # Calculation of amount to be dispensed each time.
  # n1 is a multiple of dispensed amount * total wells therefore equals total volume required
    n1 = 25*64
    for i in range(1, n1+1):
        if n1%i == 0 and i <= 300 and i%25 == 0:
            totvol = i

    tot = 0
    for j in range(1,9):
        for i in range(1,9):
            if tot == 0:
                p300.aspirate(totvol, reservoir['A7'])
                tot = totvol
            p300.dispense(25, plate2[(chr(ord('@')+j)+str(i))])
            tot -= 25
    if tot != 0:
        p300.dispense(tot, reservoir['A7'])
    p300.drop_tip()  
    p300.pick_up_tip()

    for i in range(8):
        row = plate.rows()[i]
        rowdest = plate2.rows()[i]
        p300.transfer(25, row[0:8], rowdest[0:8], new_tip='never',blow_out=True)
        p300.blow_out()
    p300.drop_tip()  

    tc_mod.close_lid() 

    tc_mod.set_block_temperature(4, hold_time_minutes=30)
    tc_mod.set_block_temperature(42, hold_time_minutes=.75)
    tc_mod.set_block_temperature(4, hold_time_minutes=5)
    tc_mod.open_lid()

    p300.pick_up_tip()
  #for i in range(8):
  #  rowdest = plate2.rows()[i]
  #  p300.transfer(150, reservoir["A8"], rowdest[0:8], new_tip='never',blow_out=True,blowout_location='destination well')
  #p300.drop_tip()
    tot = 0
    n1 = 150*64
    for i in range(1, n1+1):
        if n1%i == 0 and i <= 300 and i%150 == 0:
            totvol = i
    for j in range(1,9):
        for i in range(1,9):
            if tot == 0:
                p300.aspirate(totvol, reservoir['A8'])
                tot = totvol
            p300.dispense(150, plate2[(chr(ord('@')+j)+str(i))])
            tot -= 150

    p300.drop_tip()  
    tc_mod.close_lid() 
    tc_mod.set_block_temperature(37, hold_time_minutes=60)
    tc_mod.open_lid()